In [1]:
import sys
import os
import pandas as pd
import random

sys.path.append(os.path.abspath('../src'))

from models import Player, Team, Individual
from utils import create_valid_team, create_valid_individual, generate_initial_population
from crossover import (
    team_swap_crossover,
    player_swap_crossover,
    uniform_team_crossover,
    position_based_crossover,
    gene_level_crossover
)
from mutation import (
    random_player_mutation,
    swap_players_between_teams,
    replace_team_mutation,
    expensive_player_mutation,
    cross_position_mutation,
    forward_reset_mutation,
    rotate_teams_mutation
)
from selection import (
    tournament_selection,
    roulette_wheel_selection,
    truncation_selection,
    rank_selection,
    stochastic_universal_sampling,
    elitism,
    select_parents
)

from main import main 

In [2]:
players_csv = pd.read_csv('../data/players.csv')

In [3]:
players = []
for idx, row in players_csv.iterrows():
    p = Player(
        player_id=idx + 1,
        name=row['Name'],
        position=row['Position'],
        skill=row['Skill'],
        cost=row['Salary (€M)']
    )

    players.append(p)

print(f"Número de jogadores carregados: {len(players)}")

Número de jogadores carregados: 35


In [4]:
best_ind, history = main(players, 
                         pop_size=20, 
                         generations=30, 
                         elite_count=2,
                         selection_method="tournament",
                         crossover_rate=0.9,
                         mutation_rate=0.8,
                         verbose = False)

✅ Resultado final escrito em 'results/logs/melhor_individuo.txt'
